In [ ]:
import docker
import ipyparallel as ipp
import random
import time
import string
import hardtester
import riak

In [ ]:
##### Setup Test #####

In [ ]:
# Create Docker client
# Be sure to update the client_cert paths and base_url value
cert_path = 'path/to/cert'
key_path = 'path/to/key'
base_url = 'https://XXX.XXX.XXX.XXX:YYYY'

tls = docker.tls.TLSConfig(client_cert=(cert_path, key_path), verify=False)
docker_client = docker.Client(base_url=base_url, tls=tls)

In [ ]:
# Set repo path
# Be sure to update the path
repo_path = 'path/to/hard-tester'

In [ ]:
# Start Docker containers
hardtester.start_blockade(5, repo_path)
time.sleep(3)

In [ ]:
# Start execution cluster controller
hardtester.start_cluster_controller()
time.sleep(5)

In [ ]:
# Start execution cluster engines in each Docker container
jup_sec_path =  '~/.ipython/profile_default/security/'
hardtester.start_cluster_engines(docker_client, jup_sec_path)
time.sleep(15)

In [ ]:
# Cluster Riak and return the cluster nodes, or just get the Riak nodes
nodes = hardtester.riak_cluster(docker_client)

In [ ]:
def load_deps():
    import sys
    sys.path.append('/hardtester')
    import hardtester

In [ ]:
# Start parallel executors and import Riak in each executor engine
para_exec = hardtester.start_parallel_executor()
load_deps_res = para_exec.map(lambda node: load_deps(), nodes)
load_deps_res.result()

In [ ]:
results = list()

In [ ]:
# Set up test
bucket_type = ''.join(random.choice(string.ascii_uppercase) for _ in range(10))
bucket_name = ''.join(random.choice(string.ascii_uppercase + string.digits) for _ in range(10))
key_name = ''.join(random.choice(string.ascii_uppercase + string.digits) for _ in range(5))
value_name = ''.join(random.choice(string.ascii_uppercase + string.digits) for _ in range(5))
num_cmds_per_node = 100
timeout_ms = 2000

#bucket props
#all, one, quorum, default
props = dict()
props['name'] = bucket_type
props['allow_mult'] = 'true'
props['n_val'] = 3
props['last_write_wins'] = 'false'
props['r'] = 3
props['pr'] = 0
props['w'] = 3
props['dw'] = 3
props['pw'] = 0
props['rw'] = 3
props['notfound_ok'] = 'true'
props['basic_quorum'] = 'false'
props['consistent'] = 'false'

hardtester.create_bucket_type(docker_client, props)
time.sleep(2)
# Start tests
raw_hist = hardtester.start_consitency_test(para_exec, \
                                            nodes, \
                                            num_cmds_per_node, \
                                            bucket_type, \
                                            bucket_name, \
                                            key_name, \
                                            value_name, \
                                            timeout_ms, \
                                            repo_path)

# Parse raw history to create a causal history, order by invocation timestamp
causal_hist = hardtester.make_causal_history(raw_hist)

# Create faux Jepsen log from causal history
hardtester.make_jepsen_log(causal_hist, repo_path)

# Run linearizability checker on the faux Jepsen log
res = hardtester.run_linearizability_checker(repo_path)
results.append(res)

In [ ]:
results

In [ ]:
# Ensure there are no running execution clusters from previous runs
para_exec.shutdown(hub=True)
# Destroy Docker containers from previous runs, this will kill all running containers
hardtester.stop_blockade(repo_path)